In [1]:

import os

# This forces OpenMP to use 1 single thread, which is needed to
# prevent contention between multiple process.
os.environ['OMP_NUM_THREADS'] = '1'
# Tell numpy to only use one core.
os.environ['MKL_NUM_THREADS'] = '1'


import multiprocessing as mp
import sys
from absl import flags

import numpy as np
import torch
from torch.optim.lr_scheduler import MultiStepLR

FLAGS = flags.FLAGS

flags.DEFINE_integer('board_size', 9, 'Board size for Go.')
flags.DEFINE_float('komi', 7.5, 'Komi rule for Go.')
flags.DEFINE_integer(
    'num_stack',
    8,
    'Stack N previous states, the state is an image of N x 2 + 1 binary planes.',
)
flags.DEFINE_integer('num_res_blocks', 10, 'Number of residual blocks in the neural network.')
flags.DEFINE_integer('num_filters_resnet', 128, 'Number of filters for the conv2d layers in the neural network.')
flags.DEFINE_integer(
    'num_fc_units',
    128,
    'Number of hidden units in the linear layer of the neural network.',
)

flags.DEFINE_integer(
    'num_simulations',
    200,
    'Number of simulations per MCTS search, this applies to both self-play and evaluation processes.',
)

flags.DEFINE_integer(
    'num_parallel',
    8,
    'Number of leaves to collect before using the neural network to evaluate the positions during MCTS search,'
    '1 means no parallel search.',
)
flags.DEFINE_float(
    'c_puct_base',
    19652,
    'Exploration constants balancing priors vs. search values. Original paper use 19652',
)
flags.DEFINE_float(
    'c_puct_init',
    1.25,
    'Exploration constants balancing priors vs. search values. Original paper use 1.25',
)

flags.DEFINE_float(
    'default_rating',
    1500,
    'Default elo rating, change to the rating (for black) from last checkpoint when resume training.',
)
flags.DEFINE_string(
    'logs_dir',
    './logs/go/9x9/alphago_series',
    'Path to save statistics for self-play, training, and evaluation.',
)
flags.DEFINE_string('log_level', 'INFO', '')
flags.DEFINE_integer('seed', 1, 'Seed the runtime.')
# Initialize flags
FLAGS(sys.argv, known_only = True)

os.environ['BOARD_SIZE'] = str(FLAGS.board_size)

In [2]:
from alpha_zero.envs.go import GoEnv
from alpha_zero.core.pipeline import (
    set_seed,
    maybe_create_dir,
)
from alpha_zero.core.multi_game import run_tournament
from alpha_zero.core.quantum_net import QuantumAlphaZeroNet
from alpha_zero.core.network import AlphaZeroNet
from alpha_zero.utils.util import extract_args_from_flags_dict, create_logger

Plink failed to import tkinter.


In [3]:
agent_configs = [
    {
        'name': 'Agent_1',
        'num_filters': 128,
        'max_depth': 3,
        'branching_width': 3,
        'beam_width': 1,
        'num_fc_units':128,
        'num_search':3,
        'load_chkpt' : './checkpoints/go/9x9/quantum/d_3s_3br_3f_128be_1/training_steps_11500.ckpt'
    },

    {
        'name': 'Agent_2',
        'num_filters': 128,
        'max_depth': 5,
        'branching_width': 3,
        'beam_width': 1,
        'num_fc_units':128,
        'num_search':2,
        'load_chkpt' : './checkpoints/go/9x9/quantum/d_5s_2br_3f_128be_1/training_steps_10500.ckpt'
    },

    {
        'name': 'Agent_3',
        'num_filters': 84,
        'max_depth': 2,
        'branching_width': 3,
        'beam_width': 1,
        'num_fc_units':128,
        'num_search':5,
        'load_chkpt' : './checkpoints/go/9x9/quantum/d_2s_5br_3f_84be_1/training_steps_11500.ckpt'
    },
]


In [4]:
def env_builder():
        return GoEnv(komi=FLAGS.komi, num_stack=FLAGS.num_stack)
eval_env = env_builder()

input_shape = eval_env.observation_space.shape
num_actions = eval_env.action_space.n

# Initialize agents
Agents = []
for config in agent_configs:
    agent = QuantumAlphaZeroNet(
        input_shape,
        num_actions,
        config['num_filters'],
        config['max_depth'],
        config['branching_width'],
        config['beam_width'],
        config['num_fc_units'],
        config['num_search'],
    )
    Agents.append(agent)

resnet_agent = AlphaZeroNet(
            input_shape,
            num_actions,
            FLAGS.num_res_blocks,
            FLAGS.num_filters_resnet,
            FLAGS.num_fc_units,
        )
resnet_chkpt = './checkpoints/go/9x9/resnets/network/training_steps_5500.ckpt'

In [ ]:
# Initialize agents with metadata
agents = {
    "ResNet": {
        "network": resnet_agent,
        "elo_rating": 1200,  # Initial Elo rating
        "checkpoint": resnet_chkpt,
        "wins": 0,
        "lost": 0
    }
}

# Add QuantumAlphaZeroNet agents
for config, agent in zip(agent_configs, Agents):
    agents[config["name"]] = {
        "network": agent,
        "elo_rating": 1200,  # Initial Elo rating
        "checkpoint": config["load_chkpt"],
        "wins": 0,
        "lost":0
    }


In [6]:
import matplotlib.pyplot as plt
import os

def visualize_all_elo_ratings(agents: dict, logs_dir: str) -> None:
    """
    Create a bar chart of Elo ratings for all agents.

    Args:
        agents (dict): Dictionary containing agent information, including Elo ratings.
        logs_dir (str): Directory to save the Elo ratings chart.

    Returns:
        None
    """
    # Extract agent names and Elo ratings
    agent_names = list(agents.keys())
    elo_ratings = [agent["elo_rating"] for agent in agents.values()]

    # Create the bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(agent_names, elo_ratings, color="skyblue")
    plt.xlabel("Agents")
    plt.ylabel("Elo Rating")
    plt.title("Elo Ratings for All Agents")
    plt.xticks(rotation=45, ha="right")  # Rotate labels for better readability
    plt.tight_layout()

    # Save and display the plot
    output_path = os.path.join(logs_dir, "elo_ratings_all_agents.png")
    os.makedirs(logs_dir, exist_ok=True)
    plt.savefig(output_path)
    plt.show()

    print(f"Elo ratings chart saved at: {output_path}")


In [7]:
set_seed(FLAGS.seed)

logger = create_logger(FLAGS.log_level)

logger.info(extract_args_from_flags_dict(FLAGS.flag_values_dict()))


INFO 2024-12-02 17:37:51 3364720299.py:5] {'board_size': 9, 'komi': 7.5, 'num_stack': 8, 'num_res_blocks': 10, 'num_filters_resnet': 128, 'num_fc_units': 128, 'num_simulations': 200, 'num_parallel': 8, 'c_puct_base': 19652.0, 'c_puct_init': 1.25, 'default_rating': 1500.0, 'logs_dir': './logs/go/9x9/alphago_series', 'log_level': 'INFO', 'seed': 1}


In [8]:
if torch.cuda.is_available():
    learner_device = torch.device('cuda')

run_tournament(
    seed = FLAGS.seed,
    agents = agents,
    env = eval_env,
    device = learner_device,
    num_games = 10,
    num_simulations = FLAGS.num_simulations,
    num_parallel = FLAGS.num_parallel,
    c_puct_base = FLAGS.c_puct_base,
    c_puct_init = FLAGS.c_puct_init,
    default_rating = FLAGS.default_rating,
    log_level = FLAGS.log_level,
    logs_dir = FLAGS.logs_dir,

)

INFO 2024-12-02 17:37:51 multi_game.py:324] Starting match between ResNet and Agent_1
/home/banashree/neural-search/alpha_zero/core/multi_game.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

KeyError: 'lost'

In [ ]:
# def hook_fn(module, input, output):
#     print(f"Input shape: {module}, {input[0].shape}")  # input is a tuple; get the shape of the first element
#     print(f"Output shape:{module}, {output.shape}")

In [ ]:
# Register the hook on the first layer of conv_block1
# hook_handle = network.conv_block.register_forward_hook(hook_fn)